In [1]:
import telebot
from telebot import types
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd
from statistics import median

In [ ]:
with open('TOKEN.txt') as fh:
    token = fh.read().strip()
bot = telebot.TeleBot(token)

In [ ]:
shopping_bag = []
total_price = []

# реализация функции старт
@bot.message_handler(commands=['start'])
def show_start(message):
    bot.send_message(message.from_user.id, 'Привет! Я умею формировать список блюд на заказ из Деливери Клаб. Я помогу тебе выбрать категорию еды, ресторан из выбранной категории, а затем мы вместе сформируем список блюд для дальнейшего заказа из данного ресторана. Чтобы начать, введи команду /hungry. Чтобы посмотреть все мои функции, введи /help. Удачи!')

# предлагаем выбрать категорию еды
@bot.message_handler(commands=['hungry'])
def choose_food_category(message):
    try:
        bot.send_message(message.from_user.id, 'И так, начали! Сейчас я покажу тебе список доступных категорий еды на Деливери Клаб. ')
        global delivery
        delivery = 'https://www.delivery-club.ru'
        soup_delivery = BeautifulSoup(requests.get(delivery).text, 'lxml')
        global food_categories
        food_categories = {}
        markup = types.InlineKeyboardMarkup(row_width=2)
        buttons = []
        for line in soup_delivery.find_all('a', {'class':'cuisines-panel__slide-btn'}):
            food_categories[line.text.split()[0]] = delivery + line.get('href')
            buttons.append(types.InlineKeyboardButton(line.text.split()[0], callback_data =line.text.split()[0]))

        for i in range(0,len(buttons), 2):
            try:
                markup.row(buttons[i], buttons[i+1])
            except:
                markup.row(buttons[i])
        bot.send_message(message.from_user.id, "Выбери понравившуюся категорию.", reply_markup=markup)
    except:
        bot.send_message(message.from_user.id, "Извини, что-то полшло не так... Попробуй снова ввести команду /hungry.")

# принимаем и обрабатываем ответы пользователя с кнопочек
@bot.callback_query_handler(func=lambda call: True)

# обработка выбора категории еды, на выходе получаем список ресторанов из выбранной категории
def get_list_of_reastaurants(call):
    if call.data in list(food_categories.keys()):
        try:
            bot.send_message(call.message.chat.id, f'Через несколько секунд тебе придет файл в csv формате, в котором собраны все рестораны из категории {call.data}, а также некоторые характеристики ресторанов: среднее время доставки заказа, минимальная сумма заказа и рейтинг ресторана на основе отзывов на Деливери Клаб. Наши с тобой следующие действия:\n1) ты можешь просто выбрать свой любимый ресторан из списка всех ресторанов. Для этого введи команду /favourite;\n2) для того чтобы выбрать ресторан с относительно быстрой доставкой, нажми /min_time;\n3) для того чтобы выбрать ресторан с минимальным ограничением на сумму заказа, введи /min_check;\n4) чтобы выбрать ресторан с лучшим рейтингом, введи /best_rating.\nПрежде чем вводить команды, пожалуйста, дождись файла.')
            soup_food_category = BeautifulSoup(requests.get(food_categories[call.data]).text, 'lxml')
            global vendors
            vendors = {}
            for line, link in zip(soup_food_category.find_all('h3', {'class': 'vendor-item__title-text'}), 
                        soup_food_category.find_all('a', {'class': 'vendor-item__link'})):
                vendors[line.text.strip()] = delivery + link.get('href')

            vendors_wrap = soup_food_category.find_all('span', {'class': 'vendor-item__wrap'})
            with open('restaurants.csv', mode='w', encoding = 'cp1251') as restaurants:
                restaurants_writer = csv.writer(restaurants, delimiter=',', quotechar='"')
                restaurants_writer.writerow(['Название ресторана', 'Среднее время доставки', 'Минимальный размер чека', 'Рейтинг ресторана'])
                for i in range(len(vendors_wrap)):
                    name = vendors_wrap[i].find_all('h3', {'class': 'vendor-item__title-text'})[0].text.strip()
                    time = vendors_wrap[i].find_all('span', {'class': 'vendor-item__delivery-time'})[0].text
                    min_value = int(vendors_wrap[i].find_all('span', {'class': 'vendor-item__info-item'})[0].text.split()[1])
                    try:
                        rating = float(vendors_wrap[i].find_all('span', {'class': 'rating__value'})[0].text.replace(',' , '.'))
                    except:
                        rating = vendors_wrap[i].find_all('span', {'class': 'rating__label'})[0].text.strip()
                    restaurants_writer.writerow([name, time, min_value, rating])
            f = open('restaurants.csv', 'rb')
            bot.send_document(call.message.chat.id, f)
        except:
            bot.send_message(call.message.chat.id, "У меня не получилось собрать данные с сайта. Попробуй заново выбрать категорию еды.")
    
  # обработка названия ресторана, на выходе получаем категории из меню этого ресторана
    elif call.data in list(vendors.keys()):
        bot.send_message(call.message.chat.id, 'Отличный выбор! Сейчас я пришлю меню ресторана. Выбери одну категорию из меню ресторана.')
        try:
            soup_restaurant = BeautifulSoup(requests.get(vendors[call.data]).text, 'lxml')
            menu_sections = soup_restaurant.find_all('div', {'class': 'vendor-menu__section'})
            global titles
            titles = {}
            markup = types.InlineKeyboardMarkup()
            for section in menu_sections:
                if section.find_all('div', {'class': 'vendor-menu__section'}) == []:
                    try:
                        name = section.find_all('div', {'class': 'vendor-menu__subcategory-title'})[0].text.strip()
                        titles[name + ' product'] = section
                        markup.add(types.InlineKeyboardButton(name, callback_data=name + ' product'))
                    except:
                        name = section.find_all('div', {'class': 'vendor-menu__category-title'})[0].text.strip()
                        titles[name + ' product'] = section
                        markup.add(types.InlineKeyboardButton(name, callback_data=name + ' product'))
            
            bot.send_message(call.message.chat.id, 'Меню', reply_markup=markup)   
        except:
            bot.send_message(call.message.chat.id, 'Мне не удалось собрать информацию по этому ресторану. Попробуй выбрать другой ресторан. Извиняюсь за неудобства!')
    
   
    # обработка категории из меню ресторана, на выходе - список блюд из категории меню
    elif call.data in list(titles.keys()):
        try:
            markup = types.InlineKeyboardMarkup()
            menu_soup = titles[call.data]
            global plates_callback
            plates_callback = {}
            global prices
            prices = {}
            i = 0
            for plate in menu_soup.find_all('div', {'class': 'menu-product__info-container'}):
                plate_name = plate.find_all('div', {'class': 'menu-product__title'})[0].text
                plates_callback[f'product_{i}'] = plate_name
                plate_price = plate.find_all('div', {'class': 'menu-product__price'})[0].text.strip()
                prices[f'product_{i}'] = int(plate_price.split()[0])
                markup.add(types.InlineKeyboardButton(f'{plate_name}: Цена  {plate_price}', callback_data = f'product_{i}'))
                i += 1
            bot.send_message(call.message.chat.id, 'Пришло время сформировать заказ. Нажимая на блюдо, ты кладешь его к себе в корзину для дальнейшего заказа. Выбери блюда из одной категории, только после этого переходи к выбору блюд из другой категории меню. Чтобы просмотреть финальный заказ, нажми /order.', reply_markup=markup)
        except:
            bot.send_message(call.message.chat.id, 'У меня не полчилось получить инофрмацию об этой категории. Пожалуйста, попробуй выбрать другую категорию или дргуой ресторан.')
    
    # обработка выбора пользователем блюд из категорий меню, складываем выбранные блюда в список shopping_bag
    elif call.data.startswith('product'):        
        shopping_bag.append(plates_callback[call.data])
        total_price.append(prices[call.data])
        

 # реализации команды help   
@bot.message_handler(commands=['help'])
def show_help(message):
    bot.send_message(message.chat.id, ' /start - начало разговора с ботом.\n /hungry - бот предлагает выбрать категорию еды для заказа.\n /favourite - бот показывает список всех ресторанов в выбранной категории (вызывается после /hungry).\n /min_time - бот отображает список ресторанов в выбранной категории с самой быстрой доставкой (вызывается после /hungry).\n /min_check - бот публикует список ресторанов в выбранной категории с минимальной суммой заказа (вызывается после /hungry).\n /best_rating - бот показывает список ресторанов с лучшим рейтингом (вызывается после /hungry).\n /order - бот отображает финальный заказ.')
    

# реализация команды order, которая выводит финальный заказ и сумму заказа
@bot.message_handler(commands=['order'])
def show_final_order(message):
    bot.send_message(message.chat.id, f'Твой финальный заказ стоимостью {sum(total_price)} р.:')
    for line in shopping_bag:
        bot.send_message(message.chat.id, line)
    bot.send_message(message.chat.id, 'Спасибо за твой заказ!')
    

# реализация команды favourite, которая выводит список ресторанов из выбранной категории еды
@bot.message_handler(commands=['favourite'])
def choose_restaurant_with_min_time(message):
    rests_to_show = list(vendors.keys())
    markup = types.InlineKeyboardMarkup()
    for name in rests_to_show:
        markup.add(types.InlineKeyboardButton(name, callback_data = name))
    bot.send_message(message.chat.id, 'Публикую список всех ресторанов в выбранной категории. Выбери свой любимый.', reply_markup=markup)


# реализация команды min_time, которая выводит список ресторанов из выбранной категории еды с наименьшим временем доставки
@bot.message_handler(commands=['min_time'])
def choose_restaurant_with_min_time(message):    
    df = pd.read_csv('restaurants.csv', encoding='cp1251') 
    av_time = []
    for i in range(len(df)):
        try: 
            type(int(df.iloc[:,1][i][0])) == int
            try:
                av_time.append((int(df.iloc[:,1][i].split()[0].split('-')[0]) + int(df.iloc[:,1][i].split()[0].split('-')[1])) // 2)
            except:
                av_time.append(int(df.iloc[:,1][i].split()[0]))
        except:
            if df.iloc[:,1][i] == 'на следующий день':
                av_time.append(24*60)
            elif df.iloc[:,1][i] == 'в теч. дня':
                av_time.append(12*60)
    df['Среднее время доставки'] = av_time
    med_value_time = median(df.sort_values(axis=0, by='Среднее время доставки').iloc[:,1])
    rests_to_show = list(df[df['Среднее время доставки'] <= med_value_time]['Название ресторана'])
    markup = types.InlineKeyboardMarkup()
    for name in rests_to_show:
        markup.add(types.InlineKeyboardButton(name, callback_data = name))
    bot.send_message(message.chat.id, f'Я подсчитал, что 50% ресторанов в нашей категории доставляют заказ за {med_value_time} мин. или даже быстрее. Список самых быстрых ресторанов уже перед тобой, предлагаю тебе выбрать понравившийся.', reply_markup=markup)
 
#
# реализация команды min_check, которая выводит список ресторанов из выбранной категории еды с наименьшей минимальной суммой заказа
@bot.message_handler(commands=['min_check'])
def choose_restaurant_with_min_check(message):    
    df = pd.read_csv('restaurants.csv', encoding='cp1251') 
    min_check = min(df['Минимальный размер чека'])
    rests_to_show = list(df[df['Минимальный размер чека'] <= min_check].iloc[:,0])
    markup = types.InlineKeyboardMarkup()
    for name in rests_to_show:
        markup.add(types.InlineKeyboardButton(name, callback_data = name))
    bot.send_message(message.chat.id, f'Минимальный размер заказа - {min_check} р. Я создал подборку ресторанов, которые осущестлвяют доставку от {min_check} р. Выбери понравившийся.', reply_markup=markup)
   
#
# реализация команды best_rating, которая выводит список ресторанов из выбранной категории еды с лучшим рейтингом
@bot.message_handler(commands=['best_rating'])
def choose_restaurant_with_best_rating(message):  
    df = pd.read_csv('restaurants.csv', encoding='cp1251') 
    df.replace(to_replace=['Мало оценок', 'Новый'], value = '0', inplace = True)
    df['Рейтинг ресторана'] = list(map(float, df.iloc[:,3]))
    rests_to_show = list(df[df['Рейтинг ресторана'] >= 4.5].iloc[:,0])
    if rests_to_show != []:
        markup = types.InlineKeyboardMarkup()
        for name in rests_to_show:
            markup.add(types.InlineKeyboardButton(name, callback_data = name))
        bot.send_message(message.chat.id, f'Я подобрал для тебя лучшие рестораны с рейтингом 4.5 и выше по пятибальной шкале. Выбери ресторан из списка лучших ресторанов.', reply_markup=markup)
    else:
        bot.send_message(message.chat.id, 'Не удалось найти рестораны с высоким рейтингом. Попробуй использовать /min_time или /min_check.')
    

  

@bot.message_handler(content_types=['text'])
def get_text_message(message):
    bot.send_message(message.from_user.id, 'Я тебя не понимаю, попробуй еще раз ввести команду, либо напиши /start или /help')    
    



bot.polling()

P.S. Бот очень медленно работает. Спасибо за понимание!